# Multi-Dimensionality
todo

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.04.P23.jpg" style="width: 600px; display: inline;">

## Multiple Paths to Higher Dimensions

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.04.P21.jpg" style="width: 200px; display: inline;"><img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.04.P22.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.04.P15.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.04.P16.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.04.P17.jpg" style="width: 200px; display: inline;">

<img src="http://geometric-computation-images.s3-website-us-east-1.amazonaws.com/1.04.P18.jpg" style="width: 600px; display: inline;">

### Nests in Python

### Trees in Python

## A Glossary of Multi-Dimensional Collections